<a href="https://colab.research.google.com/github/elmarsmn/Titanic-ML/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [3]:
titanic = pd.read_csv("/content/train.csv")

In [4]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
X = titanic.drop("Survived",axis = 1)
y = titanic["Survived"].copy()
# X = titanic.drop("Name",axis = 1)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [7]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((712, 11), (179, 11), (712,), (179,))

In [8]:
cat_features = X_train.select_dtypes(exclude = [np.number]).columns
num_features = X_train.select_dtypes(include = [np.number]).columns

In [9]:
num_pipeline = Pipeline([
    ("impute",SimpleImputer(strategy = "median")),
    ("scaler",StandardScaler())
])

cat_pipeline = Pipeline([
    ("impute",SimpleImputer(strategy = "most_frequent")),
    ("one_hot",OneHotEncoder(handle_unknown = "ignore"))
])

transformer = ColumnTransformer([
    ("num_pipeline",num_pipeline,num_features)
])

In [10]:
full_pipeline = Pipeline([
    ("preprocessing",transformer),
    ("rfr",RandomForestClassifier())
])

In [11]:
full_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))])),
                ('rfr', RandomForestClassifier())])

In [12]:
full_pipeline.score(X_test,y_test)

0.7039106145251397

In [13]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV


In [14]:
param_grid = {
    "rfr__n_estimators" : [100,200,300],
    "rfr__max_depth": [10,20,None],
    "rfr__min_samples_split" : [2,10,20],
    "rfr__min_samples_leaf" : [2,5,10],
    "rfr__max_features" : ["auto","sqrt","log2"]
}

grid_search = GridSearchCV(full_pipeline,param_grid,cv = 3,scoring = "neg_mean_squared_error")
grid_search.fit(X_train[:100],y_train[:100])

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num_pipeline',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'))])),
                                       ('rfr', RandomForestClassifier())]),
             param_grid={'rfr__max_depth': [10, 20, None],
                         'rfr__max_features': ['auto', 'sqrt', 'log2'],
                         'rfr__min_samples_leaf': [2, 5, 10],
                         'rfr__min_samples_split': [2, 10, 20],
                         'rfr__n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [15]:
grid_search.best_params_

{'rfr__max_depth': 20,
 'rfr__max_features': 'sqrt',
 'rfr__min_samples_leaf': 2,
 'rfr__min_samples_split': 2,
 'rfr__n_estimators': 100}

In [16]:
best_model = grid_search.best_estimator_.fit(X_train,y_train)

In [17]:
best_model.score(X_test,y_test)

0.7374301675977654